In [ ]:
import pandas as pd
from groq import Groq
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np


In [2]:
df = pd.read_parquet("hf://datasets/Shiyu-Lab/Wikipedia_Person_Unlearn/forget_20_1/train-00000-of-00001.parquet")

In [3]:
entities = df['title'].unique().tolist()

prompt used for this generation:

For each name in the list, provide me 6 names that belong to the same domain as them (for example, if they are authors please provide authors similar as them). The output should be in a dictionary to make it into a dataframe. 

['Benedetto Varchi',
 'Wilhelm Wattenbach',
 'Elsa Triolet',
 'Theopompus',
 'Heinrich Ritter',
 'Adrienne Monnier',
 'Ann Brashares',
 'Hartmann von Aue',
 'Jorge Semprún',
 'Giovanni Battista Casti',
 'Najaf Daryabandari',
 'Heinz Erhardt',
 'Rudolf Christoph Eucken',
 'Paul Gerhardt',
 'Moshe Greenberg',
 'Amy Clampitt',
 'Ted Kooser',
 'Alfred Vogel',
 'Siegfried Lenz',
 'Philip Stanhope, 5th Earl Stanhope']

In [166]:
similar_names = {
    'Benedetto Varchi': ['Giovanni Boccaccio', 'Francesco Petrarca', 'Leonardo Bruni', 'Dante Alighieri', 'Lorenzo de Medici', 'Leon Battista Alberti'],
    'Wilhelm Wattenbach': ['Leopold von Ranke', 'Theodor Mommsen', 'Georg Waitz', 'Heinrich von Sybel', 'Friedrich Meinecke', 'Ernst Dümmler'],
    'Elsa Triolet': ['Colette', 'Simone de Beauvoir', 'Francoise Sagan', 'Nathalie Sarraute', 'Marguerite Duras', 'Anais Nin'],
    'Theopompus': ['Herodotus', 'Thucydides', 'Xenophon', 'Polybius', 'Diodorus Siculus', 'Ctesias'],
    'Heinrich Ritter': ['Immanuel Kant', 'Wilhelm von Humboldt', 'Friedrich Nietzsche', 'Arthur Schopenhauer', 'Friedrich Schelling', 'Johann Gottlieb Fichte'],
    'Adrienne Monnier': ['Sylvia Beach', 'Jane Heap', 'T.S. Eliot', 'Anna_de_Noailles', 'Djuna Barnes', 'Gertrude Stein'],
    'Ann Brashares': ['Judy Blume', 'Sara Shepard', 'Meg Cabot', 'Megan McCafferty', 'Emily Giffin', 'Jennifer Weiner'],
    'Hartmann von Aue': ['Wolfram von Eschenbach', 'Gottfried von Strassburg', 'Walter von der Vogelweide', 'Ulrich von Hutten', 'Reinmar der Alte', 'Reinmar von Hagenau'],
    'Jorge Semprún': ['Vicente Aleixandre', 'Rafael Alberti', 'Camilo Jose Cela', 'Miguel Delibes', 'Mario Vargas Llosa', 'Carlos Fuentes'],
    'Giovanni Battista Casti': ['Carlo Goldoni', 'Pietro Metastasio', 'Alessandro Scarlatti', 'Giovanni Paisiello', 'Christoph Willibald Gluck', 'Antonio Vivaldi'],
    'Najaf Daryabandari': ['Sadegh Hedayat', 'Mohammad-Taqi Bahar', 'Simin Daneshvar', 'Sadeq Chubak', 'Sohrab Sepehri', 'Bahram Beyzaie'],
    'Heinz Erhardt': ['Heinz_Rühmann', 'Dieter Hildebrandt', 'Werner Finck', 'Harald Schmidt', 'Otto Waalkes', 'Loriot'],
    'Rudolf Christoph Eucken': ['Friedrich Nietzsche', 'Arthur Schopenhauer', 'Edmund Husserl', 'Martin Heidegger', 'Karl Jaspers', 'Ernst Troeltsch'],
    'Paul Gerhardt': ['Philipp Nicolai', 'Georg Philipp Telemann', 'Nicolaus Herman', 'Johann Heermann', 'Simon Dach', 'Johann Franck'],
    'Moshe Greenberg': ['Abraham Joshua Heschel', 'Martin Buber', 'Joseph B. Soloveitchik', 'Gerhard von Rad', 'Walter Brueggemann', 'Franz Rosenzweig'],
    'Amy Clampitt': ['Elizabeth Bishop', 'Anne Sexton', 'Adrienne Rich', 'Maxine Kumin', 'Marilyn Hacker', 'Sharon Olds'],
    'Ted Kooser': ['Billy Collins', 'Rita Dove', 'Kay Ryan', 'Mark Doty', 'Joy Harjo', 'Philip_Levine_(poet)'],
    'Alfred Vogel': ['Bertolt Brecht', 'Heinrich Mann', 'Klaus Mann', 'Egon Friedell', 'Ernst Toller', 'Ernst Weiss'],
    'Siegfried Lenz': ['Heinrich Boll', 'Gunter Grass', 'Wolfgang Koeppen', 'Martin Walser', 'Peter Handke', 'Hans Magnus Enzensberger'],
    'Philip Stanhope, 5th Earl Stanhope': ['Thomas Babington Macaulay', 'Henry Hallam', 'Francis Palgrave', 'William Stubbs', 'James Anthony Froude', 'Lord Acton']
}

## Heinz earhardt - Hans_Moser_(actor)
## Heinrich Ritter - Georg Wilhelm Friedrich Hegel
## Moshe Greenberg - Elie Wiesel

In [167]:
rows = []
for key, similar_names in similar_names.items():
    for name in similar_names:
        rows.append({"title": key, "SimilarName": name})

In [168]:
new_df = pd.DataFrame(rows)

In [169]:
def get_wikipedia_url(name):
    """
    Convert a name to a Wikipedia URL-friendly format.

    Args:
        name (str): Name to convert to Wikipedia URL

    Returns:
        str: Formatted Wikipedia URL
    """
    # Replace spaces with underscores and URL encode
    formatted_name = urllib.parse.quote(name.replace(' ', '_'))
    return f'https://en.wikipedia.org/wiki/{formatted_name}'

def download_wikipedia_page(name):
    """
    Download and parse content from a Wikipedia page for a given name.

    Args:
        name (str): Name to search on Wikipedia

    Returns:
        str: Extracted content or error message
    """
    try:
        # Create Wikipedia URL
        url = get_wikipedia_url(name)

        # Send a GET request to the Wikipedia page
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)

        # Raise an exception for bad status codes
        response.raise_for_status()

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the main content (body text)
        content_div = soup.find('div', {'class': 'mw-parser-output'})

        # Extract paragraphs
        paragraphs = [p.text.strip() for p in content_div.find_all('p') if p.text.strip()]

        # Join paragraphs into a single string
        full_content = ' '.join(paragraphs)

        return full_content

    except requests.RequestException as e:
        return f"Error downloading page for {name}: {str(e)}"
    except Exception as e:
        return f"Unexpected error for {name}: {str(e)}"

def scrape_wikipedia_for_dataframe(df, name_column='SimilarName', content_column='content'):
    """
    Scrape Wikipedia for each name in the DataFrame.

    Args:
        df (pandas.DataFrame): Input DataFrame
        name_column (str): Column containing names to search
        content_column (str): Column to store Wikipedia content

    Returns:
        pandas.DataFrame: Updated DataFrame with Wikipedia content
    """
    # Create a copy of the DataFrame to avoid modifying the original
    result_df = df.copy()

    # Add content column if it doesn't exist
    if content_column not in result_df.columns:
        result_df[content_column] = ''

    # Iterate through each row
    for index, row in result_df.iterrows():
        name = row[name_column]

        try:
            # Download Wikipedia content
            content = download_wikipedia_page(name)

            # Store content in the specified column
            result_df.at[index, content_column] = content

            # Add a small delay to avoid overwhelming the server
            time.sleep(1)

            print(f"Processed {name}")

        except Exception as e:
            print(f"Error processing {name}: {str(e)}")
            result_df.at[index, content_column] = f"Error: {str(e)}"

    return result_df


In [170]:
result_df = scrape_wikipedia_for_dataframe(new_df)

Processed Giovanni Boccaccio
Processed Francesco Petrarca
Processed Leonardo Bruni
Processed Dante Alighieri
Processed Lorenzo de Medici
Processed Leon Battista Alberti
Processed Leopold von Ranke
Processed Theodor Mommsen
Processed Georg Waitz
Processed Heinrich von Sybel
Processed Friedrich Meinecke
Processed Ernst Dümmler
Processed Colette
Processed Simone de Beauvoir
Processed Francoise Sagan
Processed Nathalie Sarraute
Processed Marguerite Duras
Processed Anais Nin
Processed Herodotus
Processed Thucydides
Processed Xenophon
Processed Polybius
Processed Diodorus Siculus
Processed Ctesias
Processed Immanuel Kant
Processed Wilhelm von Humboldt
Processed Friedrich Nietzsche
Processed Arthur Schopenhauer
Processed Friedrich Schelling
Processed Johann Gottlieb Fichte
Processed Sylvia Beach
Processed Jane Heap
Processed T.S. Eliot
Processed Anna_de_Noailles
Processed Djuna Barnes
Processed Gertrude Stein
Processed Judy Blume
Processed Sara Shepard
Processed Meg Cabot
Processed Megan McCa

In [36]:
entities

['Benedetto Varchi',
 'Wilhelm Wattenbach',
 'Elsa Triolet',
 'Theopompus',
 'Heinrich Ritter',
 'Adrienne Monnier',
 'Ann Brashares',
 'Hartmann von Aue',
 'Jorge Semprún',
 'Giovanni Battista Casti',
 'Najaf Daryabandari',
 'Heinz Erhardt',
 'Rudolf Christoph Eucken',
 'Paul Gerhardt',
 'Moshe Greenberg',
 'Amy Clampitt',
 'Ted Kooser',
 'Alfred Vogel',
 'Siegfried Lenz',
 'Philip Stanhope, 5th Earl Stanhope']

In [18]:
result = df.groupby('title')['question'].count().reset_index()
result.columns = ['title', 'question_count']
print(result)

                                 title  question_count
0                     Adrienne Monnier               4
1                         Alfred Vogel               5
2                         Amy Clampitt               5
3                        Ann Brashares               3
4                     Benedetto Varchi               7
5                         Elsa Triolet               5
6              Giovanni Battista Casti               4
7                     Hartmann von Aue               5
8                      Heinrich Ritter               4
9                        Heinz Erhardt               5
10                       Jorge Semprún               6
11                     Moshe Greenberg               4
12                  Najaf Daryabandari               4
13                       Paul Gerhardt               5
14  Philip Stanhope, 5th Earl Stanhope               6
15             Rudolf Christoph Eucken               7
16                      Siegfried Lenz               8
17        

In [144]:
result_df.head()

,title,SimilarName,content
0,Benedetto Varchi,Giovanni Boccaccio,Giovanni Boccaccio (UK: /bəˈkætʃioʊ/ bə-KATCH-...
1,Benedetto Varchi,Francesco Petrarca,"Francis Petrarch (/ˈpɛtrɑːrk, ˈpiːt-/; 20 July..."
2,Benedetto Varchi,Leonardo Bruni,Leonardo Bruni[a] or Leonardo Aretino (c. 1370...
3,Benedetto Varchi,Dante Alighieri,Dante Alighieri (Italian: [ˈdante aliˈɡjɛːri];...
4,Benedetto Varchi,Lorenzo de Medici,Lorenzo di Piero de' Medici (Italian: [loˈrɛnt...


In [171]:
result_df.to_csv('domain_new.csv', index = False)

In [4]:
result_df = pd.read_csv('domain_new.csv')

In [6]:
result_df['content'].isnull().sum()

np.int64(0)

In [7]:
result_df.head()

,title,SimilarName,content
0,Benedetto Varchi,Giovanni Boccaccio,Giovanni Boccaccio (UK: /bəˈkætʃioʊ/ bə-KATCH-...
1,Benedetto Varchi,Francesco Petrarca,"Francis Petrarch (/ˈpɛtrɑːrk, ˈpiːt-/; 20 July..."
2,Benedetto Varchi,Leonardo Bruni,Leonardo Bruni[a] or Leonardo Aretino (c. 1370...
3,Benedetto Varchi,Dante Alighieri,Dante Alighieri (Italian: [ˈdante aliˈɡjɛːri];...
4,Benedetto Varchi,Lorenzo de Medici,Lorenzo di Piero de' Medici (Italian: [loˈrɛnt...


In [5]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/praveen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/praveen/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
result_df['tokenized_len'] = result_df['content'].apply(lambda x: len(word_tokenize(x)) if isinstance(x, str) else 0)
result_df['tokenized_len'].describe()

count      120.000000
mean      3106.833333
std       3498.747916
min        354.000000
25%       1065.750000
50%       1868.500000
75%       3618.750000
max      16997.000000
Name: tokenized_len, dtype: float64

In [7]:
df1_grouped = df.groupby('title')['question'].apply(list).reset_index().rename(columns={'question': 'questions'})
final_df= result_df.merge(df1_grouped, on='title', how='left')

In [8]:
final_df.head()

,title,SimilarName,content,tokenized_len,questions
0,Benedetto Varchi,Giovanni Boccaccio,Giovanni Boccaccio (UK: /bəˈkætʃioʊ/ bə-KATCH-...,2221,"[What nationality was Benedetto Varchi?, What ..."
1,Benedetto Varchi,Francesco Petrarca,"Francis Petrarch (/ˈpɛtrɑːrk, ˈpiːt-/; 20 July...",5206,"[What nationality was Benedetto Varchi?, What ..."
2,Benedetto Varchi,Leonardo Bruni,Leonardo Bruni[a] or Leonardo Aretino (c. 1370...,616,"[What nationality was Benedetto Varchi?, What ..."
3,Benedetto Varchi,Dante Alighieri,Dante Alighieri (Italian: [ˈdante aliˈɡjɛːri];...,5546,"[What nationality was Benedetto Varchi?, What ..."
4,Benedetto Varchi,Lorenzo de Medici,Lorenzo di Piero de' Medici (Italian: [loˈrɛnt...,2771,"[What nationality was Benedetto Varchi?, What ..."


In [9]:
kant = final_df[final_df['SimilarName'] == 'Immanuel Kant']

In [10]:
print(kant['content'].values[0])

Immanuel Kant[a] (born Emanuel Kant; 22 April 1724 – 12 February 1804) was a German philosopher and one of the central Enlightenment thinkers. Born in Königsberg, Kant's comprehensive and systematic works in epistemology, metaphysics, ethics, and aesthetics have made him one of the most influential and highly discussed figures in modern Western philosophy. In his doctrine of transcendental idealism, Kant argued that space and time are mere "forms of intuition" that structure all experience, and that the objects of experience are mere "appearances". The nature of things as they are in themselves is unknowable to us. Nonetheless, in an attempt to counter the philosophical doctrine of skepticism, he wrote the Critique of Pure Reason (1781/1787), his best-known work. Kant drew a parallel to the Copernican Revolution in his proposal to think of the objects of experience as conforming to our spatial and temporal forms of intuition and the categories of our understanding, so that we have a pr

### entity scraping through hyperlinks

In [4]:
df.drop_duplicates(subset=['title'], inplace=True)

In [6]:
def get_wikipedia_url(name):
    """
    Convert a name to a Wikipedia URL-friendly format.

    Args:
        name (str): Name to convert to Wikipedia URL

    Returns:
        str: Formatted Wikipedia URL
    """
    # Replace spaces with underscores and URL encode
    formatted_name = urllib.parse.quote(name.replace(' ', '_'))
    return f'https://en.wikipedia.org/wiki/{formatted_name}'

def extract_wikipedia_links(name):
    """
    Download and extract all hyperlinks from a Wikipedia page for a given name.

    Args:
        name (str): Name to search on Wikipedia

    Returns:
        list: List of hyperlinks found on the page or error message
    """
    try:
        # Create Wikipedia URL
        url = get_wikipedia_url(name)

        # Send a GET request to the Wikipedia page
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)

        # Raise an exception for bad status codes
        response.raise_for_status()

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the main content (body text)
        content_div = soup.find('div', {'class': 'mw-parser-output'})
        
        # Extract all links from the content div
        links = []
        if content_div:
            # Find all anchor tags in the content
            for a_tag in content_div.find_all('a', href=True):
                href = a_tag['href']
                
                # Filter out non-article links (Wikipedia internal links usually start with /wiki/)
                if href.startswith('/wiki/') and ':' not in href:  # Exclude special pages like File:, Category:, etc.
                    full_url = f"https://en.wikipedia.org{href}"
                    link_text = a_tag.text.strip()
                    links.append({
                        'url': full_url,
                        'text': link_text
                    })
        
        return links

    except requests.RequestException as e:
        return [{'error': f"Error downloading page for {name}: {str(e)}"}]
    except Exception as e:
        return [{'error': f"Unexpected error for {name}: {str(e)}"}]

def scrape_wikipedia_links_for_dataframe(df, name_column='title', links_column='links'):
    """
    Scrape Wikipedia links for each name in the DataFrame.

    Args:
        df (pandas.DataFrame): Input DataFrame
        name_column (str): Column containing names to search
        links_column (str): Column to store Wikipedia links

    Returns:
        pandas.DataFrame: Updated DataFrame with Wikipedia links
    """
    # Create a copy of the DataFrame to avoid modifying the original
    result_df = df.copy()

    # Add links column if it doesn't exist
    if links_column not in result_df.columns:
        result_df[links_column] = None

    # Iterate through each row
    for index, row in result_df.iterrows():
        name = row[name_column]

        try:
            # Extract Wikipedia links
            links = extract_wikipedia_links(name)

            # Store links in the specified column
            result_df.at[index, links_column] = links

            # Add a small delay to avoid overwhelming the server
            time.sleep(1)

            print(f"Processed {name}, found {len(links)} links")

        except Exception as e:
            print(f"Error processing {name}: {str(e)}")
            result_df.at[index, links_column] = [{'error': f"Error: {str(e)}"}]

    return result_df

In [7]:
entities_links = scrape_wikipedia_links_for_dataframe(df, name_column='title')

Processed Benedetto Varchi, found 32 links
Processed Wilhelm Wattenbach, found 22 links
Processed Elsa Triolet, found 165 links
Processed Theopompus, found 59 links
Processed Heinrich Ritter, found 21 links
Processed Adrienne Monnier, found 41 links
Processed Ann Brashares, found 44 links
Processed Hartmann von Aue, found 286 links
Processed Jorge Semprún, found 577 links
Processed Giovanni Battista Casti, found 32 links
Processed Najaf Daryabandari, found 533 links
Processed Heinz Erhardt, found 73 links
Processed Rudolf Christoph Eucken, found 334 links
Processed Paul Gerhardt, found 276 links
Processed Moshe Greenberg, found 49 links
Processed Amy Clampitt, found 28 links
Processed Ted Kooser, found 124 links
Processed Alfred Vogel, found 163 links
Processed Siegfried Lenz, found 368 links
Processed Philip Stanhope, 5th Earl Stanhope, found 106 links


In [13]:
entities_links.reset_index(drop=True, inplace=True)

In [17]:
entities_links['links'][18]

[{'url': 'https://en.wikipedia.org/wiki/Weimar_Republic', 'text': 'Germany'},
 {'url': 'https://en.wikipedia.org/wiki/E%C5%82k', 'text': 'Ełk'},
 {'url': 'https://en.wikipedia.org/wiki/Poland', 'text': 'Poland'},
 {'url': 'https://en.wikipedia.org/wiki/Hamburg', 'text': 'Hamburg'},
 {'url': 'https://en.wikipedia.org/wiki/Novelist', 'text': 'Novelist'},
 {'url': 'https://en.wikipedia.org/wiki/German_people', 'text': 'German'},
 {'url': 'https://en.wikipedia.org/wiki/University_of_Hamburg',
  'text': 'University of Hamburg'},
 {'url': 'https://en.wikipedia.org/wiki/The_German_Lesson',
  'text': 'The German Lesson'},
 {'url': 'https://en.wikipedia.org/wiki/Peace_Prize_of_the_German_Book_Trade',
  'text': 'Peace Prize of the German Book Trade'},
 {'url': 'https://en.wikipedia.org/wiki/Goethe_Prize', 'text': 'Goethe Prize'},
 {'url': 'https://en.wikipedia.org/wiki/List_of_honorary_citizens_of_Schleswig-Holstein',
  'text': 'Honorary citizen of Schleswig-Holstein'},
 {'url': 'https://en.wiki

In [19]:
# Example usage in Colab notebook



# Optional: Save the bidirectional links to a CSV file (one row per link)
def expand_bidirectional_links_to_dataframe(df, name_column='title', links_column='bidirectional_links'):
    """
    Expand the bidirectional links column into a new DataFrame with one link per row
    """
    rows = []
    for index, row in df.iterrows():
        name = row[name_column]
        links = row[links_column]
        
        for link in links:
            if 'error' in link:
                continue
                
            rows.append({
                'source_name': name,
                'link_text': link['text'],
                'link_url': link['url']
            })
    
    return pd.DataFrame(rows)



# Function to find more bidirectional links with fewer API calls
def find_bidirectional_links_efficient(df, name_column='title', max_links_to_check=20):
    """
    A more efficient method that only checks a subset of links
    
    Args:
        df (pandas.DataFrame): Input DataFrame
        name_column (str): Column containing names to search
        max_links_to_check (int): Maximum number of links to check per name
        
    Returns:
        dict: Dictionary mapping each name to a list of bidirectional links
    """
    result = {}
    
    for index, row in df.iterrows():
        name = row[name_column]
        print(f"Processing {name}...")
        
        # 1. Get links from the source page
        source_links = extract_wikipedia_links(name)
        print(f"  Found {len(source_links)} outgoing links")
        
        # 2. Limit the number of links to check
        links_to_check = source_links[:max_links_to_check]
        bidirectional = []
        
        # 3. Check each link for bidirectionality
        for i, link in enumerate(links_to_check):
            if 'error' in link:
                continue
                
            print(f"  Checking link {i+1}/{len(links_to_check)}: {link['text']}")
            
            # Check if the linked page references back to the source page
            if check_page_contains_reference(link['url'], name):
                bidirectional.append(link)
                print(f"    ✓ Bidirectional relationship found!")
            else:
                print(f"    ✗ No bidirectional relationship")
                
            # Add a delay to avoid overwhelming the server
            time.sleep(1)
            
        result[name] = bidirectional
        print(f"  Found {len(bidirectional)} bidirectional links for {name}")
        
    return result

# Uncomment to use the more efficient function:
# bidirectional_links = find_bidirectional_links_efficient(df, max_links_to_check=10)# Install required packages if needed
# !pip install requests beautifulsoup4 pandas

import urllib.parse
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import re

def get_wikipedia_url(name):
    """
    Convert a name to a Wikipedia URL-friendly format.

    Args:
        name (str): Name to convert to Wikipedia URL

    Returns:
        str: Formatted Wikipedia URL
    """
    # Replace spaces with underscores and URL encode
    formatted_name = urllib.parse.quote(name.replace(' ', '_'))
    return f'https://en.wikipedia.org/wiki/{formatted_name}'

def get_page_title_from_url(url):
    """
    Extract the page title from a Wikipedia URL.
    
    Args:
        url (str): Wikipedia URL
        
    Returns:
        str: Page title
    """
    # Extract the part after /wiki/
    match = re.search(r'/wiki/(.+)', url)
    if match:
        # URL decode and replace underscores with spaces
        return urllib.parse.unquote(match.group(1).replace('_', ' '))
    return None

def check_page_contains_reference(url, target_name):
    """
    Check if a Wikipedia page contains a reference to the target name.
    
    Args:
        url (str): Wikipedia URL to check
        target_name (str): Name to search for
        
    Returns:
        bool: True if the page contains a reference to the target name
    """
    try:
        # Send a GET request to the Wikipedia page
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract the main content
        content_div = soup.find('div', {'class': 'mw-parser-output'})
        if not content_div:
            return False
            
        # Find all links in the content
        for a_tag in content_div.find_all('a', href=True):
            href = a_tag['href']
            
            # Check if the link points to the target page
            if href.startswith('/wiki/'):
                link_title = get_page_title_from_url(f"https://en.wikipedia.org{href}")
                if link_title and link_title.lower() == target_name.lower():
                    return True
                    
        return False
        
    except Exception as e:
        print(f"Error checking reference in {url}: {str(e)}")
        return False

def extract_wikipedia_links(name):
    """
    Download and extract all hyperlinks from a Wikipedia page for a given name.

    Args:
        name (str): Name to search on Wikipedia

    Returns:
        list: List of hyperlinks found on the page or error message
    """
    try:
        # Create Wikipedia URL
        url = get_wikipedia_url(name)

        # Send a GET request to the Wikipedia page
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)

        # Raise an exception for bad status codes
        response.raise_for_status()

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the main content (body text)
        content_div = soup.find('div', {'class': 'mw-parser-output'})
        
        # Extract all links from the content div
        links = []
        if content_div:
            # Find all anchor tags in the content
            for a_tag in content_div.find_all('a', href=True):
                href = a_tag['href']
                
                # Filter out non-article links (Wikipedia internal links usually start with /wiki/)
                if href.startswith('/wiki/') and ':' not in href:  # Exclude special pages like File:, Category:, etc.
                    full_url = f"https://en.wikipedia.org{href}"
                    link_text = a_tag.text.strip()
                    links.append({
                        'url': full_url,
                        'text': link_text
                    })
        
        return links

    except requests.RequestException as e:
        return [{'error': f"Error downloading page for {name}: {str(e)}"}]
    except Exception as e:
        return [{'error': f"Unexpected error for {name}: {str(e)}"}]

def scrape_bidirectional_links(df, name_column='title', links_column='links', bidirectional_column='bidirectional_links', max_bidirectional=20):
    """
    Scrape Wikipedia links and identify bidirectional relationships,
    stopping after finding a specified number of bidirectional links.

    Args:
        df (pandas.DataFrame): Input DataFrame
        name_column (str): Column containing names to search
        links_column (str): Column to store all Wikipedia links
        bidirectional_column (str): Column to store bidirectional links
        max_bidirectional (int): Maximum number of bidirectional links to find per name

    Returns:
        pandas.DataFrame: Updated DataFrame with Wikipedia links
    """
    # Create a copy of the DataFrame to avoid modifying the original
    result_df = df.copy()

    # Add columns if they don't exist
    if links_column not in result_df.columns:
        result_df[links_column] = None
    if bidirectional_column not in result_df.columns:
        result_df[bidirectional_column] = None

    # First, extract all links for each name
    for index, row in result_df.iterrows():
        name = row[name_column]

        try:
            # Extract Wikipedia links
            links = extract_wikipedia_links(name)

            # Store links in the specified column
            result_df.at[index, links_column] = links

            # Add a small delay to avoid overwhelming the server
            time.sleep(1)

            print(f"Processed {name}, found {len(links)} links")

        except Exception as e:
            print(f"Error processing {name}: {str(e)}")
            result_df.at[index, links_column] = [{'error': f"Error: {str(e)}"}]

    # Now check for bidirectional relationships
    for index, row in result_df.iterrows():
        name = row[name_column]
        links = row[links_column]
        bidirectional = []

        print(f"\nChecking bidirectional links for {name}...")
        
        for i, link in enumerate(links):
            if 'error' in link:
                continue
                
            print(f"  Checking link {i+1}/{len(links)}: {link['text']}")
            
            # Check if the linked page references back to the source page
            if check_page_contains_reference(link['url'], name):
                bidirectional.append(link)
                print(f"    ✓ Bidirectional relationship found with: {link['text']}")
                
                # If we've found enough bidirectional links, stop searching
                if len(bidirectional) >= max_bidirectional:
                    print(f"    Reached maximum of {max_bidirectional} bidirectional links. Stopping search.")
                    break
            else:
                print(f"    ✗ No bidirectional relationship with: {link['text']}")
                
            # Add a delay to avoid overwhelming the server
            time.sleep(1)
            
        # Store bidirectional links in the specified column
        result_df.at[index, bidirectional_column] = bidirectional
        print(f"  Found {len(bidirectional)} bidirectional links for {name}")

    return result_df

In [20]:
result_df = scrape_bidirectional_links(df, max_bidirectional=20)

Processed Benedetto Varchi, found 32 links
Processed Wilhelm Wattenbach, found 22 links
Processed Elsa Triolet, found 165 links
Processed Theopompus, found 59 links
Processed Heinrich Ritter, found 21 links
Processed Adrienne Monnier, found 41 links
Processed Ann Brashares, found 44 links
Processed Hartmann von Aue, found 286 links
Processed Jorge Semprún, found 577 links
Processed Giovanni Battista Casti, found 32 links
Processed Najaf Daryabandari, found 533 links
Processed Heinz Erhardt, found 73 links
Processed Rudolf Christoph Eucken, found 334 links
Processed Paul Gerhardt, found 276 links
Processed Moshe Greenberg, found 49 links
Processed Amy Clampitt, found 28 links
Processed Ted Kooser, found 124 links
Processed Alfred Vogel, found 163 links
Processed Siegfried Lenz, found 368 links
Processed Philip Stanhope, 5th Earl Stanhope, found 106 links

Checking bidirectional links for Benedetto Varchi...
  Checking link 1/32: Titian
    ✗ No bidirectional relationship with: Titian
  

In [22]:
result_df.to_csv('bidirectional_links.csv', index=False)

In [ ]:
# Create expanded DataFrame with one bidirectional link per row
if any(len(row['bidirectional_links']) > 0 for _, row in result_df.iterrows()):
    expanded_df = expand_bidirectional_links_to_dataframe(result_df)
    
    # Display the first few rows
    print("\nBidirectional Links DataFrame (first 5 rows):")
    print(expanded_df.head())
    
    # Save to CSV (uncomment to use)
    # expanded_df.to_csv('wikipedia_bidirectional_links.csv', index=False)
else:
    print("\nNo bidirectional links found.")

### Q & A generation

In [89]:
system_prompt = """
You are an expert teacher, who can create questions and answers from a given context. 
Given the user wikipedia page context about {domain_person_name}, please provide as many questions and answers possible from it.
For each section, provide atleast 2 questions and answers.
The question and answers should follow the Interrogative syntactic structure, 

The questions should be on their birth, family background, education, career, achievements and other relevant topics.

The output should be in JSON format with the following keys:

{{
    "name": name of the person,
    "question1": question1,
    "answer1": answer1,
    "section" : part of the wikipedia section,
    "difficulty" : difficulty of the question,
    "question2": question2,
    .................
}}

Please be precise with the question and answer. Do not generate any other text.
"""

In [90]:
prompt = """
{content}
"""

In [ ]:
client = Groq(
    api_key = ""
)

In [ ]:
final_df['new_questions'] = ''
for i, row in final_df.iterrows():
    name = row['SimilarName']
    content = row['content']
    questions = row['questions']
    title = row['title']
    
    sys_prompt = system_prompt.format(domain_person_name=name)
    prompt_text = prompt.format(content=content)
    messages = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": prompt_text},]
    print("Now generating for ", name)
    response = client.chat.completions.create(
        messages= messages,
        model="llama-3.3-70b-versatile",
        temperature=0.1,
        max_completion_tokens=1000
    )
    response_text = response.choices[0].message.content
    final_df.at[i, 'new_questions'] = response_text
    


Now generating for  Giovanni Boccaccio
Now generating for  Francesco Petrarca
Now generating for  Leonardo Bruni
Now generating for  Dante Alighieri
Now generating for  Lorenzo de Medici
Now generating for  Leon Battista Alberti
Now generating for  Leopold von Ranke
Now generating for  Theodor Mommsen
Now generating for  Georg Waitz
Now generating for  Heinrich von Sybel
Now generating for  Friedrich Meinecke
Now generating for  Ernst Dümmler
Now generating for  Colette
Now generating for  Simone de Beauvoir
Now generating for  Francoise Sagan
Now generating for  Nathalie Sarraute
Now generating for  Marguerite Duras
Now generating for  Anais Nin
Now generating for  Herodotus
Now generating for  Thucydides
Now generating for  Xenophon
Now generating for  Polybius
Now generating for  Diodorus Siculus
Now generating for  Ctesias
Now generating for  Immanuel Kant
Now generating for  Wilhelm von Humboldt
Now generating for  Friedrich Nietzsche
Now generating for  Arthur Schopenhauer
Now ge

In [94]:
final_df.head()

,title,SimilarName,content,tokenized_len,questions,gpt_mini_tokens,new_questions
0,Benedetto Varchi,Giovanni Boccaccio,Giovanni Boccaccio (UK: /bəˈkætʃioʊ/ bə-KATCH-...,2221,"[What nationality was Benedetto Varchi?, What ...",2690,"```json\n{\n ""name"": ""Giovanni Boccaccio"",\..."
1,Benedetto Varchi,Francesco Petrarca,"Francis Petrarch (/ˈpɛtrɑːrk, ˈpiːt-/; 20 July...",5206,"[What nationality was Benedetto Varchi?, What ...",6105,"```json\n{\n ""name"": ""Francesco Petrarca"",\..."
2,Benedetto Varchi,Leonardo Bruni,Leonardo Bruni[a] or Leonardo Aretino (c. 1370...,616,"[What nationality was Benedetto Varchi?, What ...",723,"{\n ""name"": ""Leonardo Bruni"",\n ""questio..."
3,Benedetto Varchi,Dante Alighieri,Dante Alighieri (Italian: [ˈdante aliˈɡjɛːri];...,5546,"[What nationality was Benedetto Varchi?, What ...",6254,"```json\n{\n ""name"": ""Dante Alighieri"",\n ..."
4,Benedetto Varchi,Lorenzo de Medici,Lorenzo di Piero de' Medici (Italian: [loˈrɛnt...,2771,"[What nationality was Benedetto Varchi?, What ...",3127,"```\n{\n ""name"": ""Lorenzo de Medici"",\n ..."


In [95]:
def extract_qa(json_str):
    """
    Remove markdown formatting and extract questions, answers, sections, and difficulties.
    Returns a dictionary with keys: 'question', 'answer', 'section', and 'difficulty',
    where each value is a list of the corresponding items.
    """
    # Remove markdown formatting: remove starting ```json and ending ```
    json_str = json_str.strip()
    if json_str.startswith("```json"):
        json_str = json_str[len("```json"):].strip()
    if json_str.startswith("```"):
        json_str = json_str[len("```"):].strip()
    if json_str.endswith("```"):
        json_str = json_str[:-3].strip()

    # Regex pattern to extract each QA block.
    # This assumes each block is in the order: question, answer, section, difficulty.
    pattern = re.compile(
        r'"question\d+":\s*"([^"]+)",\s*'
        r'"answer\d+":\s*"([^"]+)",\s*'
        r'"section":\s*"([^"]+)",\s*'
        r'"difficulty":\s*"([^"]+)"'
    )
    matches = pattern.findall(json_str)

    # Initialize lists for each field
    questions = []
    answers = []
    sections = []
    difficulties = []

    for match in matches:
        q, a, sec, diff = match
        questions.append(q)
        answers.append(a)
        sections.append(sec)
        difficulties.append(diff)

    return {
        'question': questions,
        'answer': answers,
        'section': sections,
        'difficulty': difficulties
    }

In [96]:
qa_data = final_df['new_questions'].apply(extract_qa)
qa_df = pd.DataFrame(qa_data.tolist())

# Join the new columns to the original DataFrame
re_df = final_df.join(qa_df)

In [97]:
df_exploded = re_df.explode(['question', 'answer', 'section', 'difficulty']).reset_index(drop=True)

In [107]:
final_data = df_exploded[['title', 'SimilarName','content', 'question', 'answer', 'section', 'difficulty']]

In [103]:
qa = df_exploded.groupby('SimilarName')['question'].count().reset_index()
qa.columns = ['SimilarName', 'question_count']
print(qa)

                SimilarName  question_count
0    Abraham Joshua Heschel              16
1             Adrienne Rich               8
2      Alessandro Scarlatti              12
3                 Anais Nin              18
4          Anna_de_Noailles               7
..                      ...             ...
113    Wilhelm von Humboldt              14
114          William Stubbs               1
115        Wolfgang Koeppen               1
116  Wolfram von Eschenbach              12
117                Xenophon              12

[118 rows x 2 columns]


In [108]:
final_data.to_csv('domain_qa.csv', index = False)